# COMP9418 - Assignment 1 - Bayesian Networks as Classifiers

## UNSW Sydney, September 2021

- Ziqiao Ringgold Lin - z5324329
- Student name 2 - ZID

## Instructions

**Submission deadline:** Sunday, 17th October 2021, at 18:00:00.

**Late Submission Policy:** The penalty is set at 20% per late day. This is ceiling penalty, so if a group is marked 60/100 and they submitted two days late, they still get 60/100.

**Form of Submission:** This is a group assignment. Each group can have up to **two** students. **Only one member of the group should submit the assignment**.

You can reuse any piece of source code developed in the tutorials.

Submit your files using give. On a CSE Linux machine, type the following on the command-line:

``$ give cs9418 ass1 solution.zip``

Alternative, you can submit your solution via [WebCMS](https://webcms3.cse.unsw.edu.au/COMP9418/21T3).

## Technical prerequisites

These are the libraries your are allowed to use. No other libraries will be accepted. Make sure you are using Python 3.

In [ ]:
# Allowed libraries
import numpy as np
import pandas as pd
import scipy as sp
import heapq as pq
import matplotlib as mp
import math
from itertools import product, combinations
from graphviz import Digraph
from tabulate import tabulate
import copy

We can also use the python files we developed in tutorials, or any other code from the tutorials.

In [ ]:
from DiscreteFactors import Factor
from Graph import Graph
from BayesNet import BayesNet

## Initial task - Initialise graph

Create a graph ``G`` that represents the following network by filling in the edge lists.
![Bayes Net](BayesNet.png)


In [ ]:
G = Graph({
    "BreastDensity" : [],
    "Location" : [],
    "Age" : [],
    "BC" : [],
    "Mass" : [],
    "AD" : [],
    "Metastasis" : [],
    "MC" : [],
    "Size" : [],
    "Shape" : [],
    "FibrTissueDev" : [],
    "LymphNodes" : [],
    "SkinRetract" : [],
    "NippleDischarge" : [],
    "Spiculation" : [],
    "Margin" : [],
})

### Load data

In [ ]:
# load data
with open('bc.csv') as file:
    data = pd.read_csv(file)

#remove 2 variables from data (because we are pretending we don't know this information)
if 'Metastasis' in data:
    del data['Metastasis']
if 'LymphNodes' in data:
    del data['LymphNodes']

# remove same 2 nodes from graph
G.remove_node('Metastasis')
G.remove_node('LymphNodes')

## [20 Marks] Task 1 - Efficient d-separation test

Implement the efficient version of the d-separation algorithm in a function ``d_separation(G, X, Z, Y)`` that return a boolean: ``True`` if **X** is d-separated from **Y** given **Z** in the graph $G$ and ``False`` otherwise.

* **X**,**Y** and **Z** are python sets, each containing a set of variable names. 
* Variable names may be strings or integers, and can be assumed to be nodes of the graph $G$. 
* $G$ is a directed graph object as defined in tutorial 1.

In [ ]:
## Develop your code for d_separation(G, X, Z, Y) in one or more cells here

In [ ]:
def d_separation(G, X, Z, Y):
    '''
    Arguments:
    G:   is an object of type Graph (the class you developed in tutorial 1)
    X,Z and Y:  are python `set` objects.
    '''
    ...
    return False

In [ ]:
############
## TEST CODE
## Note: More hidden tests will be used. You should make more tests yourself.

def test(statement):
    if statement:
        print("Passed test case")
    else:
        print("Failed test case")
        
test(d_separation(G, set(['Age']), set(['BC']), set(['AD'])))
test(not d_separation(G, set(['Spiculation','SkinRetract']), set(['MC', 'Size']), set(['Age'])))

## [10 Marks] Task 2 - Estimate Bayesian Network parameters from data

Implement a function ``learn_outcome_space(data)`` that learns the outcome space (the valid values for each variable) from the pandas dataframe ``data`` and returns a dictionary ``outcomeSpace`` with these values.

Implement a method ``model.learn_parameters(data, alpha=1)`` that learns the parameters of the Bayesian Network `model`. This function should do the same as the ``learn_parameters`` function from tutorials, but it should also implement laplacian smoothing with parameter $\alpha$.

In [ ]:
## Develop your code for learn_outcome_space(data) in one or more cells here

In [ ]:
def learn_outcome_space(data):
    '''
    Arguments:
        data - A pandas dataframe
    Returns: 
        outcomeSpace - A dictionary. e.g. {'A':('True', 'False'), 'B':('up','down','left'), 'C':(1,2,3,4)}
    '''
    ...
    return outcomeSpace


outcomeSpace = learn_outcome_space(data)

In [ ]:
############
## TEST CODE

outcomeSpace = learn_outcome_space(data)

outcomes = outcomeSpace['BreastDensity']
answer = ('high', 'medium', 'low')
test(len(outcomes) == len(answer) and set(outcomes) == set(answer))

In [ ]:
## Develop your code for learnParameters in one or more cells here

In [ ]:
class BayesNet(BayesNet):
    def learn_parameters(self, data, alpha=1):
        ...
...
model = BayesNet(...)

In [ ]:
############
## TEST CODE

model.learn_parameters(data, alpha=1)

test(model.factors['Age']['35-49'] == 0.248000399920016)

## [20 Marks] Task 3 - Bayesian Network Classification

Design a new function ``assess_bayes_net(model, dataframe, var)`` that uses the test cases in ``dataframe`` to assess the performance of the Bayesian network at classifying the variable `var`. Implement the efficient classification procedure discussed in the lectures (make sure you understand what a Markov Blanket is). Such a function should return the classifier accuracy. 

 * ``var`` is the name of the variable you are predicting, using the values of all the other variables. 
 
If you like, you can add new functions to the BayesNet class, or write helper functions to help solve the above task.

Using another function called `cross_validation_bayes_net`, compute and report the average accuracy over the ten cross-validation runs as well as the standard deviation. A scaffold for this function is provided below.

In [ ]:
## Develop your code for assess_bayes_net in one or more cells here

In [ ]:
model = BayesNet(...)
def assess_bayes_net(model, dataframe, var='BC'):
    ...
    return acc

def cross_validation_bayes_net(dataframe, var='BC', k=10):
    accuracy_list = []
    for i in range(k):
        # split dataset into train and test
        ...
        
        # train a model
        ...
        
        # test the model with assess_bayes_net
        acc = ...
        
        accuracy_list.append(acc)
    return np.mean(accuracy_list), np.std(accuracy_list)


In [ ]:
############
## TEST CODE

acc, stddev = cross_validation_bayes_net(data, 'BC', 10)
test(abs(acc - 0.85) < 0.05)

## [10 Marks] Task 4 - Naïve Bayes Classification

Design a new function ``assess_naive_bayes(model, data, var)`` to classify and assess the test cases in ``data``. To classify each example, use the log probability trick discussed in the lectures. Do $k$-fold cross-validation with the `cross_validation_naive_bayes(data, var, k)` function, same as above, and return ``acc`` and ``stddev``.

In [ ]:
## Develop your code for assess_naive_bayes(model, data, var) in one or more cells here

In [ ]:
...


def assess_naive_bayes(model, dataframe, var='BC'):
    ...
    return accuracy
    
def cross_validation_naive_bayes(dataframe, var='BC', k=10):
    accuracy_list = []
    for i in range(k):
        # split dataset into train and test
        ...
        
        # create and train a model
        ...
        
        # test the model with assess_naive_bayes
        acc = ...
        
        accuracy_list.append(acc)
    return np.mean(accuracy_list), np.std(accuracy_list)
    

In [ ]:
############
## TEST CODE

acc, stddev = cross_validation_naive_bayes(data, 'BC')
test(abs(acc - 0.80) < 0.05)

## [20 Marks] Task 5 - Tree-augmented Naïve Bayes Classification

Similarly to the previous task, implement a Tree-augmented Naïve Bayes (TAN) classifier and evaluate your implementation in the breast cancer dataset. Design a function ``learn_tan_structure(data, class_var)`` to learn the TAN structure (graph) from ``data`` and return such a structure. Scaffolds for required functions are given below. Implement other helper functions as necessary.

In [ ]:
## Develop your code for learn_tan_structure(data) in one or more cells here

In [ ]:
def learn_tan_structure(data, class_var='BC'):
    '''
    Arguments:
        data: a dataframe
        class_var: The variable you will be classifying with this graph structure
    Return:
        graph: A Graph object
    '''
    ...
    return graph

...

def cross_validation_tan(data, var='BC', k=10):
    ...
    return np.mean(accuracy_list), np.std(accuracy_list)

In [ ]:
############
## TEST CODE

tan_graph = learn_tan_structure(data)
test(len(tan_graph.children('BC')) == len(tan_graph)-1)
test('FibrTissueDev' in tan_graph.children('Spiculation') or 'Spiculation' in tan_graph.children('FibrTissueDev'))

## [20 Marks] Task 6 - Report

Write a report (**with less than 500 words**) summarising your findings in this assignment. Your report should address the following:

a. Make a summary and discussion of the experimental results. You can analyse your results from different aspects such as accuracy, runtime, coding complexity and independence assumptions. You can use plots to illustrate your results.

b. Discuss the time and memory complexity of the implemented algorithms.

Use Markdown and Latex to write your report in the Jupyter notebook. Develop some plots using Matplotlib to illustrate your results. Be mindful of the maximum number of words. Please, be concise and objective.